# Кросс валидация

In [ ]:
 Создаем DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)  # Добавили метки для тестовой выборки

# Задаем параметры
params = {
    'objective': 'reg:squarederror',
    'booster': 'gblinear'  
}
trees = 1000

# Кросс-валидация
cv = xgb.cv(params, dtrain, metrics='rmse', nfold=10, num_boost_round=trees, verbose_eval=False)

# Обучение модели с оптимальным числом деревьев
best_n_estimators = cv['test-rmse-mean'].argmin()
bst = xgb.train(params, dtrain, num_boost_round=best_n_estimators,
                evals=[(dtrain, 'train'), (dtest, 'eval')],  # Добавили eval_set
                early_stopping_rounds=50) # Добавили early_stopping

# Прогноз на обучающем наборе
prediction_train = bst.predict(dtrain)
#  Оценка качества модели на обучающем наборе
rmse_train = np.sqrt(mean_squared_error(y_train, prediction_train)) # Исправили ошибку
mae_train = mean_absolute_error(y_train, prediction_train) # Исправили ошибку
print(f'RMSE (Train): {rmse_train}')
print(f'MAE (Train): {mae_train}')

# Прогноз на тестовом наборе
prediction_test = bst.predict(dtest)
# Оценка качества модели на тестовом наборе
rmse_test = np.sqrt(mean_squared_error(y_test, prediction_test)) # Исправили ошибку
mae_test = mean_absolute_error(y_test, prediction_test) # Исправили ошибку
print(f'RMSE (Test): {rmse_test}')
print(f'MAE (Test): {mae_test}')

In [ ]:
# График для тестового набора
plt.figure(figsize=(15, 5))
plt.plot(prediction_test, label='Predicted (Test)', color='green')
plt.plot(lower, 'r--', label='Lower Bound', color='red')
plt.plot(upper, 'r--', label='Upper Bound', color='red')
plt.plot(y_test.values, label='Actual (Test)', color='orange')
plt.title(f"XGBoost Mean Absolute Error: {round(mean_absolute_error(prediction_test, y_test), 2)}")
plt.xlabel('Samples')
plt.ylabel('Values')
plt.legend(loc='best')
plt.grid(True)
plt.tight_layout()
plt.show()